In [3]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense

In [4]:
data = pd.read_csv("train.txt", sep=';')
data.columns = ["Text", "Emotions"]
print(data.head())

                                                Text Emotions
0  i can go from feeling so hopeless to so damned...  sadness
1   im grabbing a minute to post i feel greedy wrong    anger
2  i am ever feeling nostalgic about the fireplac...     love
3                               i am feeling grouchy    anger
4  ive been feeling a little burdened lately wasn...  sadness


In [6]:
texts = data["Text"].tolist()
labels=data["Emotions"].tolist()

#Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [7]:
sequence = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequence])
padded_sequences = pad_sequences(sequence, maxlen = max_length)

In [8]:
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [10]:
one_hot_labels = keras.utils.to_categorical(labels)

In [11]:
x_train, x_test, y_train, y_test = train_test_split(padded_sequences, one_hot_labels, test_size=0.2)

In [15]:
model= Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1,
                    output_dim=128,
                    input_length = max_length))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=len(one_hot_labels[0]), activation='softmax'))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, epochs = 10, batch_size = 32, validation_data=(x_test, y_test))

Epoch 1/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 14s 31ms/step - accuracy: 0.3853 - loss: 1.5120 - val_accuracy: 0.6844 - val_loss: 0.8824
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 20s 29ms/step - accuracy: 0.8760 - loss: 0.4085 - val_accuracy: 0.8138 - val_loss: 0.5841
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 22s 33ms/step - accuracy: 0.9858 - loss: 0.0559 - val_accuracy: 0.8181 - val_loss: 0.5996
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 14s 35ms/step - accuracy: 0.9945 - loss: 0.0189 - val_accuracy: 0.8091 - val_loss: 0.6492
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 24s 43ms/step - accuracy: 0.9977 - loss: 0.0129 - val_accuracy: 0.8153 - val_loss: 0.6670
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 18s 45ms/step - accuracy: 0.9981 - loss: 0.0083 - val_accuracy: 0.8009 - val_loss: 0.7326
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 18s 39ms/step - accuracy: 0.9980 - loss: 0.0093 - val_accuracy: 0.8125 - val_loss: 0.7085
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 18s 32ms/step - accuracy: 0.9984 - loss: 0.0073 - 

In [17]:
input_text = "She didn't come today because she lost her dog yesterday!"

#Preprocess  the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
['sadness']
